In [1]:
%pylab inline
import numpy as np
import pandas
import os
import re
import sklearn
import scipy
import calendar

Populating the interactive namespace from numpy and matplotlib


In [2]:
convert_to_string=lambda k : str(k-1)
convert=lambda k : str(k)
returnweekday = lambda k : calendar.weekday(int(k[:4]),int(k[5:7]),int(k[8:10]))

In [3]:
poi_agglomerated=pandas.read_table('poi_agglomerated',index_col=0,sep=',')
poi_reduced_agglomerated=pandas.read_table('poi_reduced_agglomerated',index_col=0)
poi_reduced_sum=pandas.read_table('poi_reduced_summed',index_col=0)
poi_reduced_probadded=pandas.read_table('poi_reduced_probadded',index_col=0)
poi_importance=pandas.read_table('importance_agglomeration',index_col=0)

In [4]:
# os.chdir('..')
test_data=pandas.read_table('./test_set/cleaned_complete',index_col=0)
test_poi=pandas.read_table('./test_set/compressed_poi',index_col=0)

In [5]:
os.chdir('./training_data')

In [6]:
labels=pandas.read_table('labels',index_col=0)

In [7]:
# order_data=pandas.read_table('final_data_',index_col=0)

In [8]:
order_data=pandas.read_table('final_data_complete',index_col=0)
train_poi=pandas.read_table('compressed_poi',index_col=0)

In [9]:
order_data.drop(labels=order_data.columns[3:-5],axis=1,inplace=True)

In [10]:
order_data=order_data.join(train_poi,on='start_region_hash')

In [11]:
# order_data.drop('week_day',axis=1,inplace=True)

In [12]:
order_data=order_data.astype(np.float64)

In [13]:
val_match=pandas.read_table('validation_set',index_col=0)

In [14]:
val_match=val_match[val_match.columns[[0,1,-2,-1]]]

In [15]:
sir_val_match=pandas.read_table('val_match',index_col=0)

In [16]:
validation_set=order_data[order_data.day>18]


In [17]:
sample_data=test_data.copy()

In [18]:
sample_data.insert(len(sample_data.columns)-2,'week_day',sample_data.date.apply(returnweekday))

In [19]:
sample_data.date=sample_data.time_slot

In [20]:
sample_data.drop(labels=sample_data.columns[3:-6],axis=1,inplace=True)

In [21]:
sample_data=sample_data.astype(np.float64)

In [22]:
sample_data.drop('time_slot',axis=1,inplace=True)

In [23]:
sample_data=sample_data.join(test_poi,on='start_region_hash')

In [24]:
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.preprocessing import OneHotEncoder as ohe
from sklearn.preprocessing import LabelEncoder as le
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import FeatureHasher
from category_encoders.polynomial import PolynomialEncoder
from category_encoders.hashing import HashingEncoder

/home/rameez/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [25]:
# order_data.weather=sample_data.weather.apply(np.ceil)
# order_data.weather=order_data.weather.apply(np.ceil)
# order_data.start_region_hash=order_data.start_region_hash.apply(convert_to_string)
# order_data.date=order_data.date.apply(convert_to_string)
# order_data.week_day=order_data.week_day.apply(convert_to_string)
# order_data.weather=order_data.weather.apply(convert_to_string)


sample_data.weather=sample_data.weather.apply(np.ceil)
# sample_data.start_region_hash=sample_data.start_region_hash.apply(convert_to_string)
# sample_data.date=sample_data.date.apply(convert_to_string)
# sample_data.week_day=sample_data.week_day.apply(convert_to_string)
# sample_data.weather=sample_data.weather.apply(convert_to_string)

In [ ]:
def add_poi(poi_agglomerated,poi_reduced_sum,poi_reduced_agglomerated,poi_reduced_probadded,poi_importance,comb_type,order_data,sample_data,validation_set):    
    if comb_type=='complete_agglomerated':
        order_data=order_data.join(poi_agglomerated,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        sample_data=sample_data.join(poi_agglomerated,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        validation_set=validation_set.join(poi_agglomerated,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
    elif comb_type=="reduced_sum":
        order_data=order_data.join(poi_reduced_sum,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        sample_data=sample_data.join(poi_reduced_sum,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        validation_set=validation_set.join(poi_reduced_sum,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
    elif comb_type=='reduced_agglomerated':
        order_data=order_data.join(poi_reduced_agglomerated,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        sample_data=sample_data.join(poi_reduced_agglomerated,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        validation_set=validation_set.join(poi_reduced_agglomerated,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
    elif comb_type=='reduced_probadded':
        order_data=order_data.join(poi_reduced_probadded,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        sample_data=sample_data.join(poi_reduced_probadded,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        validation_set=validation_set.join(poi_reduced_probadded,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
    elif comb_type=='importance':
        order_data=order_data.join(poi_importance,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        sample_data=sample_data.join(poi_importance,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
        validation_set=validation_set.join(poi_importance,on='start_region_hash',lsuffix='_caller',rsuffix='_other')
    return order_data,sample_data,validation_set

In [ ]:
order_data,sample_data,validation_set=add_poi(poi_agglomerated,poi_reduced_sum,poi_reduced_agglomerated,poi_reduced_probadded,poi_importance,'importance',order_data,sample_data,validation_set)

In [ ]:
one_toeighteen=order_data[order_data.day<19]

In [ ]:
one_toeighteen_labels=labels.gap.values[:one_toeighteen.shape[0]]

In [ ]:
one_toeighteen.shape,one_toeighteen_labels.shape

In [ ]:

minmax_scaler=StandardScaler()
minmax_scaler.fit(np.array(order_data.values,dtype=np.float64))

In [ ]:
order_data[order_data.columns]=minmax_scaler.transform(np.array(order_data.values,dtype=np.float64))

In [ ]:
validation_set[validation_set.columns]=minmax_scaler.transform(np.array(validation_set.values,dtype=np.float64))

In [ ]:
one_toeighteen[one_toeighteen.columns]=minmax_scaler.transform(np.array(one_toeighteen.values,dtype=np.float64))

In [ ]:
sir_val_match[sir_val_match.columns[1:]]=minmax_scaler.transform(np.concatenate((np.array(sir_val_match.values,dtype=float64),np.zeros((sir_val_match.shape[0],4))),axis=1))[:,1:4]

### One Hot Encoding

In [ ]:
r=np.arange(len(order_data.columns))
oh=ohe(n_values=[66,144,13,7],categorical_features=r[[0,1,-2,-1]],sparse=False)

In [ ]:
order_data_encoded=oh.fit_transform(order_data.values)

In [ ]:
order_data_encoded.shape

In [ ]:
order_sparse=scipy.sparse.csr_matrix(order_data_encoded)

In [ ]:
sample_data[sample_data.columns]=minmax_scaler.transform(np.array(sample_data.values,dtype=np.float64))

In [ ]:
sample_data_encoded=oh.fit_transform(sample_data.values)

In [ ]:
sample=convert_to_polynomial(sample_data)

In [ ]:
order_data.weather.shape

In [ ]:
def hash_feature(values,encode_type):
    hashed=None
    if encode_type=='date':
        hashed=date_hasher.transform(values)
    elif encode_type=='days':
        hashed=days_hasher.transform(values)
    elif encode_type=='weather':
        hashed=weather_hasher.transform(values)
    else:
        hashed=region_hasher.transform(values)
    return hashed

In [ ]:
order=convert_to_polynomial(order_data)

In [ ]:
sample=convert_to_polynomial(sample_data)

In [ ]:
sample

In [ ]:
date_pe=PolynomialEncoder(cols=['date'])
days_pe=PolynomialEncoder(cols=['week_day'])
weather_pe=PolynomialEncoder(cols=['weather'])
region_pe=PolynomialEncoder(cols=['start_region_hash'])

# date_pe=date_pe.fit(pandas.DataFrame(order_data.date.apply(convert_to_string)))
# days_pe=days_pe.fit(pandas.DataFrame(order_data.week_day.apply(convert_to_string)))
# weather_pe=weather_pe.fit(pandas.DataFrame(pandas.DataFrame(order_data.weather.apply(convert_to_string))))
# region_pe=region_pe.fit(pandas.DataFrame(pandas.DataFrame(order_data.start_region_hash.apply(convert_to_string))))

date_pe=date_pe.fit(order_data)
days_pe=days_pe.fit(order_data)
weather_pe=weather_pe.fit(order_data)
region_pe=region_pe.fit(order_data)

In [ ]:
def convert_to_polynomial(order_data,sparse=False):
    order_copy=order_data.copy()
#     if hashed:
#     date_hashed=date_pe.transform(pandas.DataFrame(order_data.date.apply(convert_to_string),columns=['date']))
#     days_hashed=days_pe.transform(pandas.DataFrame(order_data.week_day.apply(convert_to_string)),columns=['week_days'])
#     weather_hashed=weather_pe.transform(pandas.DataFrame(order_data.weather.apply(convert_to_string)),columns['weather'])
#     region_hashed=region_pe.transform(pandas.DataFrame(order_data.start_region_hash.apply(convert_to_string)),columns=['region'])


    date_hashed=date_pe.transform(order_data)
    days_hashed=days_pe.transform(order_data)
    weather_hashed=weather_pe.transform(order_data)
    region_hashed=region_pe.transform(order_data)
    return pandas.concat([region_hashed,order_data[order_data.columns[2:-2]],weather_hashed,days_hashed,date_hashed],axis=1)


#     else:
#         date_sparse=one_hot_encode(np.array(order_data.date.values.reshape((-1,1)),dtype=int),'date')
#         region_sparse=one_hot_encode(np.array(order_data.start_region_hash.values.reshape((-1,1)),dtype=int),'region')
#         weather_sparse=one_hot_encode(np.array(order_data.weather.values.reshape((-1,1)),dtype=int),'weather')
#         weekday_sparse=one_hot_encode(np.array(order_data.week_day.values.reshape((-1,1)),dtype=int),'days')

#     Y=np.array(labels.values.ravel(),dtype=np.float64)
#     print Y
    if sparse:
        remaining_sparse=scipy.sparse.csr_matrix(np.array(order_data.values[:,2:-2],dtype=np.float64))
        complete_data=scipy.sparse.hstack([region_sparse,date_sparse,remaining_sparse,weather_sparse,weekday_sparse])
#     labels_sparse=scipy.sparse.csr_matrix(Y.reshape((-1,1)))
    else:
        region_hashed
        return pandas.concat([region_hashed,date_hashed,pandas.DataFrame(order_data.values[:,2:-2]),weather_hashed,days_hashed],axis=1,ignore_index=True)
#         return pandas.DataFrame({region_hashed.values,date_hashed.values,weather_hashed.values,days_hashed.values})

In [ ]:
def convert_to_sparse(order_data,sparse=True):
    date_sparse=None
    region_sparse=None
    weather_sparse=None
    weekday_sparse=None
#     if hashed:
#     date_sparse=hash_feature(order_data.date.apply(convert_to_string).values,'date')
#     region_sparse=hash_feature(order_data.start_region_hash.apply(convert_to_string).values,'region')
#     weather_sparse=hash_feature(order_data.weather.apply(convert_to_string).values,'weather')
#     weekday_sparse=hash_feature(order_data.week_day.apply(convert_to_string).values,'days')
#     else:
    date_sparse=one_hot_encode(np.array(order_data.date.values.reshape((-1,1)),dtype=int),'date')
    region_sparse=one_hot_encode(np.array(order_data.start_region_hash.values.reshape((-1,1)),dtype=int),'region')
    weather_sparse=one_hot_encode(np.array(order_data.weather.values.reshape((-1,1)),dtype=int),'weather')
    weekday_sparse=one_hot_encode(np.array(order_data.week_day.values.reshape((-1,1)),dtype=int),'days')

#     Y=np.array(labels.values.ravel(),dtype=np.float64)
#     print Y
    if sparse:
        remaining_sparse=scipy.sparse.csr_matrix(np.array(order_data.values[:,2:-2],dtype=np.float64))
        complete_data=scipy.sparse.hstack([region_sparse,date_sparse,remaining_sparse,weather_sparse,weekday_sparse])
#     labels_sparse=scipy.sparse.csr_matrix(Y.reshape((-1,1)))
        return complete_data
    else:
        f=pandas.DataFrame(region_sparse.toarray())
        g=pandas.DataFrame(date_sparse.toarray())
        h=pandas.DataFrame(weather_sparse.toarray())
        i=pandas.DataFrame(weekday_sparse.toarray())
        j=pandas.DataFrame(order_data.values[:,2:-2],columns=order_data.columns[2:-2])
        print f.shape,g.shape,j.shape,h.shape,i.shape
        return pandas.concat([f,g,j,h,i],axis=1,ignore_index=True).values

In [ ]:
def convertData_for_randomforest(order_data,predict=False):
    data_copy=order_data.copy()
    if predict:
        data_copy.start_region_hash=data_copy.district_date_slot.apply(convert_to_string)        
    else:
        data_copy.start_region_hash=data_copy.start_region_hash.apply(convert_to_string)
    data_copy.date=data_copy.date.apply(convert_to_string)
    data_copy.weather=data_copy.weather.apply(convert_to_string)
    data_copy.week_day=data_copy.week_day.apply(convert_to_string)
    return data_copy

In [ ]:
def randomforest_error(model,X,Y):
    print X.shape[0]
    pclass=model.predict(X)
    d=pandas.DataFrame(X.to_dense())
    d['gap']=np.abs(Y-pclass)
    grouped=d.groupby('start_region_hash')
    summed=grouped.gap.sum()
    counts=grouped.gap.count()
    return np.mean(summed/counts)
#     Y=pandas.DataFrame(Y,dtype=np.float64)
#     ytest.gap=ytest.gap.values-pclasses
#     ytest.gap=ytest.gap.abs()
#     ytest.drop(labels=['date'],axis=1,inplace=True)
#     grouped=ytest.groupby(by='start_region_hash')
#     innercounts=grouped.count()
#     innersum=grouped.sum()
#     innervals=innersum/innercounts
#     return np.mean(innervals.gap.values)
    

# Creating Submission File

In [269]:
os.chdir('..')
sample=pandas.read_table('sample.csv',sep=',')
os.chdir('./training_data')
sample

,id,gap
0,08232402614a9b48895cc3d0aeb0e9f2_2016-01-23_0,0
1,08f5b445ec6b29deba62e6fd8b0325a6_2016-01-23_0,0
2,0a5fef95db34383403d11cb6af937309_2016-01-23_0,0
3,1afd7afbc81ecc1b13886a569d869e8a_2016-01-23_0,0
4,1c60154546102e6525f68cb4f31e0657_2016-01-23_0,0
5,1cbfbdd079ef93e74405c53fcfff8567_2016-01-23_0,0
6,1ecbb52d73c522f184a6fc53128b1ea1_2016-01-23_0,0
7,2301bc920194c95cf0c7486e5675243c_2016-01-23_0,0
8,2350be163432e42270d2670cb3c02f80_2016-01-23_0,0
9,2407d482f0ffa22a947068f2551fe62c_2016-01-23_0,0


In [270]:
# sample.set_index('district_date_slot',inplace=True)

In [271]:
# sample.reset_index(inplace=True)

In [272]:
func = lambda k: re.findall(r'_(.*)',k)[0]
returndistrict= lambda k : re.findall(r'(.*?)_.*',k)[0]

In [273]:
testing=test_data.copy()

In [274]:
testing['gap']=predicted

In [275]:
testing.time_slot=testing.time_slot.apply(convert)

In [276]:
testing.date=testing.date+'_'+testing.time_slot

In [277]:
testing.drop('time_slot',axis=1,inplace=True)

In [278]:
# testing.drop(['poi_info','temperature','pm25','weather','week_day'],axis=1,inplace=True)
testing.drop(labels=testing.columns[2:-1],axis=1,inplace=True)

In [279]:
os.chdir('..')
cluster_map=pandas.read_table('./test_set/cluster_map/cluster_map',sep='\t',names=['region_hash','region_id'])
os.chdir('./training_data')

In [280]:
cluster_map.set_index('region_id',inplace=True)

In [281]:
addunder=lambda k : str(k)+'*'
addnew=lambda k : str(k)+'_'

In [282]:
seperate_hash=lambda k : str(re.findall(r'(.*?)_',k)[0])

In [283]:
testing.start_region_hash=testing.start_region_hash.replace(cluster_map.index,cluster_map.region_hash)

In [284]:
testing.start_region_hash=testing.start_region_hash+'_'+testing.date

In [285]:
testing.drop('date',axis=1,inplace=True)

In [286]:
# testing.join(sample,on='start_region_hash',how='right',lsuffix='_caller',rsuffix='_other')
temp=sample.join(testing.set_index('start_region_hash'),on='id',lsuffix='_caller',rsuffix='_other')

In [287]:
temp.gap_other.nonzero()

(array([    0,     1,     2, ..., 47517, 47518, 47519]),)

In [288]:
temp.gap_other=temp.gap_other.fillna(value=0.0)

In [289]:
temp.gap_other.nonzero()

(array([ 2775,  2776,  2777, ..., 47318, 47319, 47320]),)

In [290]:
temp.drop('gap_caller',axis=1,inplace=True)


In [291]:
temp=temp.rename(columns={'gap_other':'gap'})

In [292]:
temp.set_index('id',inplace=True)

In [293]:
temp.to_csv('new_try_submission.csv')

In [ ]:
testing

# Applying Algorithms

In [26]:
from sklearn.learning_curve import validation_curve
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression

/home/rameez/anaconda2/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [27]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

In [27]:
def rms_error(model, X, y):
#     print model.feature_importances_
    y_pred = model.predict(X)
    return np.mean(np.abs(y - y_pred))

In [ ]:
115000

In [ ]:
def plot_with_err(x, data, **kwargs):
    mu, std = data.mean(1), data.std(1)
    print mu,std
    lines = plt.plot(x, mu, '-', **kwargs)
    plt.fill_between(x, mu - std, mu + std, edgecolor='none',
                     facecolor=lines[0].get_color(), alpha=0.2)

def plot_it(degrees,val_train,val_test):
    plot_with_err(degrees, val_train, label='training scores')
    plot_with_err(degrees, val_test, label='validation scores')
    plt.xlabel('degree'); plt.ylabel('rms error')
    plt.legend();

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
l=Lasso()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(order_data.values, labels.gap.values, test_size=0.3)

In [ ]:
all_imp=pandas.read_table('importance',index_col=0)
# feature_importance=pandas.read_table('best50',index_col=0)

In [ ]:
feats=all_imp.values

In [ ]:
feats=feats.ravel()

In [ ]:
np.where(order_data.columns=='poi_info')[0].tolist()

In [ ]:
feats=feats[-20:]

In [ ]:
order_data.columns[feats]

In [ ]:
pf=PolynomialFeatures(degree=2)

In [ ]:
higher_order=pf.fit_transform(order_data.values[:,feats[-1:]])

In [ ]:
higher_sample=pf.transform(sample_data.values[:,feats[-1:]])

In [ ]:
higher_validation=pf.transform(validation_set.values[:,feats[-20:]])

In [ ]:
finalfeats=np.concatenate(([0,1],feats))
finalfeats

In [ ]:
feature_importance.to_csv('importance',sep='\t')

In [ ]:
feature_importance.to_csv('best50',sep='\t')

In [ ]:
np.argsort(forest.feature_importances_),forest.feature_importances_
feature_importance=pandas.DataFrame(np.argsort(forest.feature_importances_),columns=['feature_importance'])

# Random Forest Regressor

In [29]:
forest=rfr(n_estimators=800,max_depth=9,min_impurity_split=0.1,oob_score=True,verbose=2,n_jobs=-1)
forest.fit(X_train,y_train)

building tree 1 of 2000building tree 2 of 2000

building tree 3 of 2000
building tree 4 of 2000
building tree 5 of 2000
building tree 6 of 2000
building tree 7 of 2000
building tree 8 of 2000
building tree 9 of 2000
building tree 10 of 2000
building tree 11 of 2000
building tree 12 of 2000
building tree 13 of 2000
building tree 14 of 2000
building tree 15 of 2000
building tree 16 of 2000
building tree 17 of 2000
building tree 18 of 2000
building tree 19 of 2000
building tree 20 of 2000
building tree 21 of 2000
building tree 22 of 2000
building tree 23 of 2000
building tree 24 of 2000
building tree 25 of 2000
building tree 26 of 2000
building tree 27 of 2000
building tree 28 of 2000
building tree 29 of 2000
building tree 30 of 2000
building tree 31 of 2000
building tree 32 of 2000
building tree 33 of 2000
building tree 34 of 2000
building tree 35 of 2000
building tree 36 of 2000


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.6s


building tree 37 of 2000
building tree 38 of 2000
building tree 39 of 2000
building tree 40 of 2000
building tree 41 of 2000
building tree 42 of 2000
building tree 43 of 2000
building tree 44 of 2000
building tree 45 of 2000
building tree 46 of 2000
building tree 47 of 2000
building tree 48 of 2000
building tree 49 of 2000
building tree 50 of 2000
building tree 51 of 2000
building tree 52 of 2000
building tree 53 of 2000
building tree 54 of 2000
building tree 55 of 2000
building tree 56 of 2000
building tree 57 of 2000
building tree 58 of 2000
building tree 59 of 2000
building tree 60 of 2000
building tree 61 of 2000
building tree 62 of 2000
building tree 63 of 2000
building tree 64 of 2000
building tree 65 of 2000
building tree 66 of 2000
building tree 67 of 2000
building tree 68 of 2000
building tree 69 of 2000
building tree 70 of 2000
building tree 71 of 2000
building tree 72 of 2000
building tree 73 of 2000
building tree 74 of 2000
building tree 75 of 2000
building tree 76 of 2000


[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   24.8s


building tree 159 of 2000
building tree 160 of 2000
building tree 161 of 2000
building tree 162 of 2000
building tree 163 of 2000
building tree 164 of 2000
building tree 165 of 2000
building tree 166 of 2000
building tree 167 of 2000
building tree 168 of 2000
building tree 169 of 2000
building tree 170 of 2000
building tree 171 of 2000
building tree 172 of 2000
building tree 173 of 2000
building tree 174 of 2000
building tree 175 of 2000
building tree 176 of 2000
building tree 177 of 2000
building tree 178 of 2000
building tree 179 of 2000
building tree 180 of 2000
building tree 181 of 2000
building tree 182 of 2000
building tree 183 of 2000
building tree 184 of 2000
building tree 185 of 2000
building tree 186 of 2000
building tree 187 of 2000
building tree 188 of 2000
building tree 189 of 2000
building tree 190 of 2000
building tree 191 of 2000
building tree 192 of 2000
building tree 193 of 2000
building tree 194 of 2000
building tree 195 of 2000
building tree 196 of 2000
building tre

...

building tree 1454 of 2000
building tree 1455 of 2000
building tree 1456 of 2000
building tree 1457 of 2000
building tree 1458 of 2000
building tree 1459 of 2000
building tree 1460 of 2000
building tree 1461 of 2000
building tree 1462 of 2000
building tree 1463 of 2000
building tree 1464 of 2000
building tree 1465 of 2000
building tree 1466 of 2000
building tree 1467 of 2000
building tree 1468 of 2000
building tree 1469 of 2000
building tree 1470 of 2000
building tree 1471 of 2000
building tree 1472 of 2000
building tree 1473 of 2000
building tree 1474 of 2000
building tree 1475 of 2000
building tree 1476 of 2000
building tree 1477 of 2000
building tree 1478 of 2000
building tree 1479 of 2000
building tree 1480 of 2000
building tree 1481 of 2000
building tree 1482 of 2000
building tree 1483 of 2000
building tree 1484 of 2000
building tree 1485 of 2000
building tree 1486 of 2000
building tree 1487 of 2000
building tree 1488 of 2000
building tree 1489 of 2000
building tree 1490 of 2000
b

[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  5.4min


building tree 1982 of 2000
building tree 1983 of 2000
building tree 1984 of 2000
building tree 1985 of 2000
building tree 1986 of 2000
building tree 1987 of 2000
building tree 1988 of 2000
building tree 1989 of 2000
building tree 1990 of 2000
building tree 1991 of 2000
building tree 1992 of 2000
building tree 1993 of 2000
building tree 1994 of 2000
building tree 1995 of 2000
building tree 1996 of 2000
building tree 1997 of 2000
building tree 1998 of 2000
building tree 1999 of 2000
building tree 2000 of 2000


[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  5.4min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=0.1, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
           oob_score=True, random_state=None, verbose=2, warm_start=False)

In [30]:
vt=rms_error(forest,X_test,y_test)
vt

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 1005 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Done 1450 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done 1977 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 2000 out of 2000 | elapsed:    4.1s finished


3.5882722498129058

In [31]:
et=rms_error(forest,X_train,y_train)
et

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done 1005 tasks      | elapsed:    3.7s
[Parallel(n_jobs=4)]: Done 1450 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done 1977 tasks      | elapsed:    7.3s
[Parallel(n_jobs=4)]: Done 2000 out of 2000 | elapsed:    7.4s finished


3.2609750205486074

In [32]:
vt-et

0.32729722926429838

In [33]:
predicted=forest.predict(sample_data.values)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 1005 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 1450 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 1977 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done 2000 out of 2000 | elapsed:    2.4s finished


In [34]:
predicted.sum()

111864.37644057749

In [ ]:
def validate_importance(order_data,labels,feats,validation_set,val_match):
    train_vals=[]
    test_vals=[]
    validation_err=[]
    for i in np.arange(19,feats.shape[0],5):
        trees=np.arange(180,190,10)
        val_train, val_test = validation_curve(rfr(oob_score=True,random_state=120,min_impurity_split=0,n_jobs=-1), order_data.values[:,feats[i:]] , labels.gap.values,
                                           'n_estimators', trees,verbose=2, cv=4,scoring=rms_error)
        train_vals.append(np.mean(val_train))
        test_vals.append(np.mean(val_test))
        forest=rfr(n_estimators=180,oob_score=True,random_state=120,min_impurity_split=0,n_jobs=-1)
        forest.fit(order_data.values[:,feats[i:]] , labels.gap.values)
        err=rms_error(forest,validation_set.values[:,feats[i:]],val_match.gap.values)
        validation_err.append(err)
        print 'feats : ',feats.shape[0]-i,' error : ',np.mean(val_test),' validation_err : ',err
    return train_vals,test_vals,validation_err

In [ ]:
validate_importance(order_data,labels,feats,validation_set,val_match)

In [ ]:
tup=([0.84959129192507854,
  0.84965859730545124,
  0.84962453681301375,
  0.84951406564718979,
  0.84904609257464092,
  0.85015506929785301,
  0.84959820168838018,
  0.85027037714894038,
  0.84947607412320703,
  0.84970895616763098,
  0.84968034468699716,
  0.85000287854297396,
  0.8502126885450445,
  0.84994359275888898,
  0.84985683958718672,
  0.85008813610474077,
  0.84946724851373623,
  0.84895111428276993,
  0.84940803167777945,
  0.84914020563794679,
  0.84903657618860162,
  0.84914259669496972,
  0.84912288728943075,
  0.84959042755920522,
  0.84924206682144388,
  0.84917646177815143,
  0.84900497442958556,
  0.84923298456949048,
  0.84942107213474372,
  0.84949202572162263,
  0.84992714683090365,
  0.85167763150800857,
  0.85882583396487633,
  0.92436531326215365,
  1.8665943613716736],
 [6.9640800210176987,
  6.9341999708416822,
  6.9656849773103051,
  6.968308281818218,
  6.9483994650316818,
  6.9188789788249334,
  6.9372629867440789,
  6.9177321578005904,
  6.7597060361974748,
  6.6814963210342704,
  6.6883786756016246,
  6.6752724006110578,
  6.6991621962342993,
  6.7338273855849557,
  6.7162685986369013,
  6.6771242128637773,
  6.75378682163009,
  6.6319163507292007,
  6.6257908549709654,
  6.5983683180195758,
  6.5403235696226858,
  6.5208086982211881,
  6.5455128160864975,
  6.7436158090270721,
  6.7753279501296877,
  6.423858400079391,
  6.3777005575033643,
  6.421873677834574,
  6.463348634251691,
  6.6073033486628869,
  6.3385582415194861,
  6.3207376808670777,
  7.2505689773196123,
  6.9299889230273219,
  6.4057110618092628],
 [1.2044112727289862,
  1.2043523740864821,
  1.2061862422194893,
  1.2017975865692248,
  1.2032316507169143,
  1.2079147995796993,
  1.2062698782918448,
  1.2042119597227523,
  1.2044541509407294,
  1.2048798703287487,
  1.2064138265741251,
  1.2041641340250389,
  1.2053821579520236,
  1.2037341739347589,
  1.2039471514260538,
  1.2083487647776694,
  1.204237403936314,
  1.2029913442554976,
  1.2026445490484334,
  1.2025255737905751,
  1.2043292858186205,
  1.2039308954007228,
  1.2049166230816712,
  1.2086828378779528,
  1.2043745199760636,
  1.2051899127828902,
  1.2035553576561167,
  1.2001201532307084,
  1.2080757106710203,
  1.2067658048617296,
  1.2065942920147574,
  1.2070975220163125,
  1.2157822646576657,
  1.3104396915732135,
  2.6481216179134699])

In [ ]:
train_err=tup[0]
test_err=tup[1]
validation_err=tup[2]

In [ ]:
ranges=np.arange(19,feats.shape[0],5)
plt.plot(ranges,train_err)
plt.plot(ranges,test_err)
plt.plot(ranges,validation_err)

In [ ]:
np.argmin(train_err),np.argmin(test_err),np.argmin(validation_err)

In [ ]:
feats.shape[0]-ranges[31],feats.shape[0]-ranges[27]

In [ ]:
temp=validation_set.copy()
temp['actual']=labels.gap.values[-validation_set.shape[0]:]
te=val_match.join(temp.set_index(['start_region_hash','day','date']),['start_region_hash','day','date'])
te.drop(te.columns[1:-1],inplace=True,axis=1)

In [ ]:
def err(model,testing,val_match):
    val_matches=val_match.copy()
    pclasses=model.predict(testing.values)
    test=testing.copy()
    test['predicted']=pclasses
    new=test.set_index(['start_region_hash','day','date'])
    val_matches=val_matches.join(new,['start_region_hash','day','date'])
    val_matches.predicted=val_matches.predicted.fillna(0.0)
    return np.mean(np.abs(val_matches.predicted.values-val_matches.gap.values))

In [ ]:
val_match=val_match.astype(np.float64)


In [ ]:
err(forest,validation_set,pandas.DataFrame(val_match))

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr=LinearRegression(fit_intercept=True,n_jobs=-1)
lr.fit(order_data.values[:,finalfeats],labels.gap.values,forest.feature_importances_)

In [ ]:
rms_error(lr,validation_set.values,val_match.gap.values)

In [ ]:
def predict(order_data,validation_set,val_match):
    forest=rfr(n_estimators=320,oob_score=True,random_state=120,min_impurity_split=0,verbose=2,n_jobs=-1)
    forest.fit(order_data.values,labels.gap.values)
#     print err(forest,validation_set,pandas.DataFrame(val_match))
    print rms_error(forest,validation_set.values,val_match.gap.values)

In [ ]:
predict(order_data,validation_set,val_match)

In [ ]:
plot_it(trees,val_train,val_test)

In [ ]:
from sklearn.linear_model import

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
arrayform=convert_to_sparse(order_data,sparse=False)

In [ ]:
pca=PCA(n_components=10)
pca.fit(arrayform)

In [ ]:
n=pca.transform(arrayform)

In [ ]:
from sklearn.cluster import FeatureAgglomeration

In [ ]:
FA=FeatureAgglomeration(n_clusters=100,n_components=5,affinity='cosine',linkage='complete')
FA.fit(order_data_encoded)

In [ ]:
transformed=FA.transform(order_data_encoded)

In [ ]:
transformed

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
L=Lasso(max_iter=2000)

In [ ]:
L.fit(order_data.values,labels.gap.values)

In [ ]:
L.coef_

In [ ]:
v=np.where(L.coef_>0.0)

In [ ]:
v[0]

In [ ]:
pp=L.predict(order_data_encoded)

In [ ]:
np.mean(np.abs(labels.gap.values-pp))

In [ ]:
def lasso_err(model,X,Y):
    p=model.predict(X)
    return np.mean(np.abs(p-Y))

In [ ]:
alphas

In [ ]:
alphas = np.arange(100,5000,200)
iters=np.append([5000]*6,np.arange(10000,23000,4000))
val_train, val_test = validation_curve(Lasso(max_iter=10000), order_data_encoded , labels.gap.values,
                                       'alpha', alphas,verbose=2, cv=4,scoring=lasso_err)


In [ ]:
vt=np.mean(val_train,axis=1)
vtest=np.mean(val_test,axis=1)

In [ ]:
val_train.shape
val_test


In [ ]:
def plot_with_err(x, data, **kwargs):
    mu, std = data.mean(1), data.std(1)
    print mu,std,x
    lines = plt.plot(x, mu, '-', **kwargs)
    plt.fill_between(x, mu - std, mu + std, edgecolor='none',
                     facecolor=lines[0].get_color(), alpha=0.2)

plot_with_err(alphas, val_train, label='training scores')
plot_with_err(alphas, val_test, label='validation scores')
plt.xlabel('degree'); plt.ylabel('rms error')
plt.legend();

In [ ]:
plt.plot(alphas,vt[:vt.shape[0]])
plt.plot(alphas,vtest[:vt.shape[0]])


In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(order_data.values, labels.gap.values, test_size=0.3)

In [ ]:
SGD=SGDRegressor(n_iter=500,epsilon=0.5,penalty='l1',loss='huber',verbose=2,eta0=0.01,alpha=0.01)
SGD.fit(order_data.values,labels.gap.values)


In [ ]:
ptrain=SGD.predict(X_train[:,v[0]])

In [ ]:
ptest=SGD.predict(X_test[:,v[0]])

In [ ]:
print 'train : ',np.mean(np.abs(y_train-ptrain))
print 'test  : ',np.mean(np.abs(y_test-ptest))

In [ ]:
npredicted=SGD.predict(sample_data_encoded[:,v[0]])

In [ ]:
from sklearn.svm import SVR

In [ ]:
degree = np.arange(3,10,1)
val_train, val_test = validation_curve(SVR(max_iter=20000), order_data_encoded , labels.gap.values,
                                       'degree', degree,verbose=2,n_jobs=2, cv=4,scoring=lasso_err)


In [ ]:
ptest=svr.predict(X_test[:,v[0]])

In [ ]:
print 'test  : ',np.mean(np.abs(y_test-ptest))

In [ ]:
predicted=svr.predict(sample_data_encoded[:,v[0]])

# Multi Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(order_data.values, labels.gap.values, test_size=0.3)

In [ ]:
mlp=MLPRegressor(hidden_layer_sizes=(500,3),activation='relu',solver='adam',alpha=0.01,learning_rate_init=0.1,verbose=2)
mlp.fit(one_toeighteen,one_toeighteen_labels)

In [ ]:
err(mlp,validation_set,val_match)

In [ ]:
rms_error(mlp,validation_set.values,val_match.gap.values)

In [ ]:
mlp.n_layers_

In [ ]:
np.mean(np.abs(mlp.predict(X_test)-y_test))

In [ ]:
order_data

In [ ]:
def predict(order_data,validation_set,val_match):
    forest=rfr(n_estimators=180,oob_score=True,random_state=120,min_impurity_split=0,verbose=2,n_jobs=-1)
    forest.fit(order_data.values,labels.gap.values)
    print err(forest,validation_set,pandas.DataFrame(val_match))
    print rms_error(forest,validation_set.values,val_match.gap.values)

In [ ]:
predict(order_data,validation_set,val_match)

# XGBOOST REGRESSOR

In [28]:
# from xgboost import XGBRegressor
from xgboost.sklearn import XGBRegressor
import xgboost as xgb

In [89]:
X_train, X_test, y_train, y_test = train_test_split(order_data.values, labels.gap.values, test_size=0.3)

In [ ]:
newfeats=feats[-50:]

In [ ]:
order_data.columns[best9]

In [ ]:
best9=[2,1,180,0,179,183,182,181,3]

In [230]:
xg=XGBRegressor(n_estimators=2500,max_depth=8,seed=2,silent=False,reg_lambda=0.2,scale_pos_weight=0.5,learning_rate=0.016,gamma=0.5)
# xg.fit(one_toeighteen.values[:,feats],one_toeighteen_labels,eval_metric='mae',verbose=True)
xg.fit(order_data.values,labels.gap.values)
# xg.fit(order_data.values[:,best9],labels.gap.values)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0.5, learning_rate=0.016, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=2500, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=0.2,
       scale_pos_weight=0.5, seed=2, silent=False, subsample=1)

In [193]:
rms_error(xg,validation_set.values,val_match.gap.values)

1.9574159963289384

In [194]:
xg.score(X_train,y_train)

0.99766641095375075

In [195]:
te=rms_error(xg,X_train,y_train)
te

1.3657968331515364

In [196]:
ve=rms_error(xg,X_test,y_test)
ve

2.1982597479017372

In [197]:
ve-te

0.83246291475020073

In [231]:
predicted=xg.predict(sample_data.values)

In [233]:
predicted.sum()

111724.39

In [ ]:
def pipe_(X_train,y_train,X_test,y_test):
    new_train=X_train
    new_test=X_test
    for i in xrange(2):
        xg=XGBRegressor(n_estimators=600,max_depth=9,silent=False,reg_lambda=0.25,scale_pos_weight=0.5,learning_rate=0.05,gamma=0.55)
        # xg.fit(one_toeighteen.values[:,feats],one_toeighteen_labels,eval_metric='mae',verbose=True)
        xg.fit(new_train,y_train)
        xg.score(new_train,y_train)
        te=rms_error(xg,new_train,y_train)
        ve=rms_error(xg,new_test,y_test)
        p=xg.predict(new_train)
        pt=xg.predict(new_test)
        new_train=np.concatenate((X_train,p.reshape((-1,1))),axis=1)
        new_test=np.concatenate((X_test,pt.reshape((-1,1))),axis=1)
        print te,ve,ve-te
    return pt

In [ ]:
np.concatenate(([2,3],[4,5]))

In [ ]:
pipe_(X_train[:,best9],y_train,X_test[:,best9],y_test)

In [ ]:
lambdas = np.arange(40, 80,10)
val_train, val_test = validation_curve(XGBRegressor(n_estimators=1500,max_depth=10,silent=False), order_data.values[:,best9] , labels.gap.values,
                                       'reg_lambda', lambdas, cv=3, scoring=rms_error, verbose=2)


In [ ]:
plot_it(lambdas,val_train,val_test)

In [ ]:
np.mean(val_test,axis=1)

# XGB FEATURE IMPORTANCES

In [ ]:
# fig,ax=plt.subplots(1,1,figsize=(200,200))
ax=xgb.plot_importance(xg,height=0.5,grid=False)
fig=ax.figure
fig.set_size_inches(20,60)

In [ ]:
fig.savefig('feature_importances_XGB.jpg')

# STACKING

In [29]:
from sklearn.cross_validation import KFold

In [262]:
order_data.drop(['day','weather'],axis=1,inplace=True)


In [267]:
sample_data.drop(['day','weather'],axis=1,inplace=True)


In [263]:
class Ensemble(object):
    def __init__(self, n_folds, stacker, base_models):
        self.n_folds = n_folds
        self.stacker = stacker
        self.base_models = base_models
    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        folds = list(KFold(len(y), n_folds=self.n_folds, shuffle=True, random_state=2016))
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], len(folds)))
            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                # y_holdout = y[test_idx]
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]
            S_test[:, i] = S_test_i.mean(1)
        self.stacker.fit(S_train, y)
        y_pred = self.stacker.predict(S_test)[:]
        return y_pred

In [264]:
rfr3=rfr(n_estimators=600,max_depth=12,min_impurity_split=0.1,oob_score=True,verbose=2,n_jobs=-1)
rfr4=rfr(n_estimators=400,max_depth=9,min_impurity_split=0,oob_score=True,verbose=2,n_jobs=-1)
rfr1=rfr(n_estimators=800,max_depth=10,min_impurity_split=0.1,oob_score=True,verbose=2,n_jobs=-1)
rfr2=rfr(n_estimators=1200,max_depth=11,min_impurity_split=0.1,oob_score=True,verbose=2,n_jobs=-1)


forest=rfr(n_estimators=800,max_depth=9,min_impurity_split=0.1,oob_score=True,verbose=2,n_jobs=-1)


In [265]:
es=Ensemble(4,forest,[rfr1,rfr2,rfr3,rfr4])

In [268]:
predicted=es.fit_predict(order_data.values,labels.gap.values,sample_data.values)

building tree 1 of 800
building tree 2 of 800
building tree 3 of 800
building tree 4 of 800
building tree 5 of 800
building tree 6 of 800
building tree 7 of 800
building tree 8 of 800
building tree 9 of 800
building tree 10 of 800
building tree 11 of 800
building tree 12 of 800
building tree 13 of 800
building tree 14 of 800
building tree 15 of 800
building tree 16 of 800
building tree 17 of 800
building tree 18 of 800
building tree 19 of 800
building tree 20 of 800
building tree 21 of 800
building tree 22 of 800
building tree 23 of 800
building tree 24 of 800
building tree 25 of 800
building tree 26 of 800
building tree 27 of 800
building tree 28 of 800
building tree 29 of 800
building tree 30 of 800
building tree 31 of 800
building tree 32 of 800
building tree 33 of 800
building tree 34 of 800
building tree 35 of 800
building tree 36 of 800
building tree 37 of 800
building tree 38 of 800


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.2s


building tree 39 of 800
building tree 40 of 800
building tree 41 of 800
building tree 42 of 800
building tree 43 of 800
building tree 44 of 800
building tree 45 of 800
building tree 46 of 800
building tree 48 of 800
building tree 47 of 800
building tree 49 of 800
building tree 50 of 800
building tree 51 of 800
building tree 52 of 800
building tree 53 of 800
building tree 54 of 800
building tree 55 of 800
building tree 56 of 800
building tree 57 of 800
building tree 58 of 800
building tree 59 of 800
building tree 60 of 800
building tree 61 of 800
building tree 62 of 800
building tree 63 of 800
building tree 64 of 800
building tree 65 of 800
building tree 66 of 800
building tree 67 of 800
building tree 68 of 800
building tree 69 of 800
building tree 70 of 800
building tree 71 of 800
building tree 72 of 800
building tree 73 of 800
building tree 74 of 800
building tree 75 of 800
building tree 76 of 800
building tree 77 of 800
building tree 78 of 800
building tree 79 of 800
building tree 80

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   18.9s


building tree 159 of 800
building tree 160 of 800
building tree 161 of 800
building tree 162 of 800
building tree 163 of 800
building tree 164 of 800
building tree 165 of 800
building tree 166 of 800
building tree 167 of 800
building tree 168 of 800
building tree 169 of 800
building tree 170 of 800
building tree 171 of 800
building tree 172 of 800
building tree 173 of 800
building tree 174 of 800
building tree 175 of 800
building tree 176 of 800
building tree 177 of 800
building tree 178 of 800
building tree 179 of 800
building tree 180 of 800
building tree 181 of 800
building tree 182 of 800
building tree 183 of 800
building tree 184 of 800
building tree 185 of 800
building tree 186 of 800
building tree 187 of 800
building tree 188 of 800
building tree 189 of 800
building tree 190 of 800
building tree 191 of 800
building tree 192 of 800
building tree 193 of 800
building tree 194 of 800
building tree 195 of 800
building tree 196 of 800
building tree 197 of 800
building tree 198 of 800


...

[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.6min


building tree 363 of 800
building tree 364 of 800
building tree 365 of 800
building tree 366 of 800
building tree 367 of 800
building tree 368 of 800
building tree 369 of 800
building tree 370 of 800
building tree 371 of 800
building tree 372 of 800
building tree 373 of 800
building tree 374 of 800
building tree 375 of 800
building tree 376 of 800
building tree 377 of 800
building tree 378 of 800
building tree 379 of 800
building tree 380 of 800
building tree 381 of 800
building tree 382 of 800
building tree 383 of 800
building tree 384 of 800
building tree 385 of 800
building tree 386 of 800
building tree 387 of 800
building tree 388 of 800
building tree 389 of 800
building tree 390 of 800
building tree 391 of 800
building tree 392 of 800
building tree 393 of 800
building tree 394 of 800
building tree 395 of 800
building tree 396 of 800
building tree 397 of 800
building tree 398 of 800
building tree 399 of 800
building tree 400 of 800
building tree 401 of 800
building tree 402 of 800


[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.9min


building tree 645 of 800
building tree 646 of 800
building tree 647 of 800
building tree 648 of 800
building tree 649 of 800
building tree 650 of 800
building tree 651 of 800
building tree 652 of 800
building tree 653 of 800
building tree 654 of 800
building tree 655 of 800
building tree 656 of 800
building tree 657 of 800
building tree 658 of 800
building tree 659 of 800
building tree 660 of 800
building tree 661 of 800
building tree 662 of 800
building tree 663 of 800
building tree 664 of 800
building tree 665 of 800
building tree 666 of 800
building tree 667 of 800
building tree 668 of 800
building tree 669 of 800
building tree 670 of 800
building tree 671 of 800
building tree 672 of 800
building tree 673 of 800
building tree 674 of 800
building tree 675 of 800
building tree 676 of 800
building tree 677 of 800
building tree 678 of 800
building tree 679 of 800
building tree 680 of 800
building tree 681 of 800
building tree 682 of 800
building tree 683 of 800
building tree 684 of 800


[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  3.6min finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 800 out of 800 | elapsed:    0.8s finished


In [41]:
np.mean(np.abs(predicted-y_test))

2.219400187506634

In [ ]:
forest=rfr(n_estimators=2500,max_depth=9,oob_score=True,min_impurity_split=0,verbose=2,n_jobs=-1)
